In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp
import mosek
import matplotlib.pyplot as plt
import scipy.stats
import phi_divergence as phi
from time import process_time


In [2]:
def toymodel(Z_arr):
    x = cp.Variable(2, nonneg = True)
    constraints = [Z_arr @ x <= 1, x<= 10]
    obj = cp.Maximize(cp.sum(x))
    prob = cp.Problem(obj,constraints)
    prob.solve(solver=cp.MOSEK)
    return(x.value)

def lowbound(x,p,r, par, phi_div):
    q = cp.Variable(2, nonneg = True)
    constraints = [cp.sum(q) == 1]
    constraints = phi_div(p,q,r,par,constraints)
    obj = cp.Minimize(q[0])
    prob = cp.Problem(obj,constraints)
    prob.solve(solver=cp.MOSEK)
    return(prob.value)

In [12]:
par = 1
phi_div = phi.kb_cut
n = 1000
phi_dot = 1
r = phi_dot/(2*n)*scipy.stats.chi2.ppf(0.99, 1)

In [13]:
np.random.seed(1)

In [14]:
data = np.random.uniform(-1,1,size = (n,2))
Z_arr = np.array([data[0],data[1]])
x = toymodel(Z_arr)
print('true prob', 1/2+1/(2*x[1]))
cons_check = data.dot(x) <= 1
feas = np.where(cons_check == True)
infeas = np.where(cons_check == False)
p = np.array([len(feas[0])/n,len(infeas[0])/n])


true prob 0.5828424631268384


In [15]:
lowbound(x,p,r, par, phi_div)

0.5053660487429287

In [16]:
p

array([0.546, 0.454])